In [56]:
from pulp import *
# initialize the model
prob = LpProblem("multiPerFinPLan", LpMaximize)

cash_flow = {
             0 :  20,
             1 :  80,
             2 :  90,
             3 :  70,
             4 :  80,
             }
payment_flow = {
             0 :  0,
             1 :  60,
             2 :  70,
             3 :  50,
             4 :  50,
             }
credit_rate1 = .02
credit_rate2 = .04
bond_yield = .03
bond_maturity = 2
invest_rate = .01
T = len(cash_flow)
# ---------------------
# VARIABLES
# ---------------------

cashinhand = LpVariable.dicts("cashinhand", range(1, T), 0, None)
bond = LpVariable.dicts("bond", range(0, T), 0, None) 
payment1 = LpVariable.dicts("payment1", range(1, T), 0, None)
payment2 = LpVariable.dicts("payment2", range(1, T +1), 0, None) # extra period to match second constraint
payment3 = LpVariable.dicts("payment3", range(1, T + 2), 0, None)# extra period to match second constraint

# Objective function
prob += cashinhand[T-1], "Objective"

# Constraints
prob += payment2[1] == 0
prob += payment2[T] == 0

prob += payment3[1] == 0
prob += payment3[2] == 0
prob += payment3[T] == 0
prob += payment3[T+1] == 0

prob += bond[0] == 0
prob += bond[3] == 0
prob += bond[4] == 0

prob += payment1[1] + cashinhand[1] + bond[1] == 100

for i in range(2, T):
    prob += payment1[i] + cashinhand[i] + bond[i] + payment2[i] + payment3[i] - (1 + invest_rate) * cashinhand[i - 1] - \
    (1 + bond_yield) * bond[i - bond_maturity] == cash_flow[i]

for i in range(1, T):
    prob += payment1[i] +  payment2[i + 1] * (1/(1 + credit_rate1))  + payment3[i + 2] * (1 / (1 + credit_rate2)) == payment_flow[i]

prob.writeLP("multiPerFinPLan.lp")
    
status = prob.solve(GLPK(options=["--ranges","multiPerFinPLan.sen"]))
print(status)

#print the result
print("Cashinhand")
for i in range(1, T):
    print(cashinhand[i].value())


print("bond")       
for i in range(1, T):
    print(bond[i].value())

print("P1")   
for i in range(1, T):
    print(payment1[i].value())
    
print("P2")    
for i in range(1, T):
    print(payment2[i].value())


print("P3")    
for i in range(1, T):
    print(payment2[i].value())
    
    
print("Objective", value(prob.objective))

1
Cashinhand
0.0
0.0
61.2
112.412
bond
40.0
20.0
0.0
0.0
P1
60.0
70.0
50.0
50.0
P2
0.0
0.0
0.0
0.0
P3
0.0
0.0
0.0
0.0
Objective 112.412


In [52]:
prob


multiPerFinPLan:
MAXIMIZE
1*cashinhand_4 + 0
SUBJECT TO
_C1: payment2_1 = 0

_C2: payment2_5 = 0

_C3: payment3_1 = 0

_C4: payment3_2 = 0

_C5: payment3_5 = 0

_C6: payment3_6 = 0

_C7: bond_0 = 0

_C8: bond_3 = 0

_C9: bond_4 = 0

_C10: bond_1 + cashinhand_1 + payment1_1 = 100

_C11: - 1.03 bond_0 + bond_2 - 1.01 cashinhand_1 + cashinhand_2 + payment1_2
 + payment2_2 + payment3_2 = 90

_C12: - 1.03 bond_1 + bond_3 - 1.01 cashinhand_2 + cashinhand_3 + payment1_3
 + payment2_3 + payment3_3 = 70

_C13: - 1.03 bond_2 + bond_4 - 1.01 cashinhand_3 + cashinhand_4 + payment1_4
 + payment2_4 + payment3_4 = 80

_C14: payment1_1 + 0.980392156863 payment2_2 + 0.961538461538 payment3_3 = 60

_C15: payment1_2 + 0.980392156863 payment2_3 + 0.961538461538 payment3_4 = 70

_C16: payment1_3 + 0.980392156863 payment2_4 + 0.961538461538 payment3_5 = 50

_C17: payment1_4 + 0.980392156863 payment2_5 + 0.961538461538 payment3_6 = 50

VARIABLES
bond_0 Continuous
bond_1 Continuous
bond_2 Continuous
bond_3 Co

In [ ]:
# %load multiPerFinPLan.sen
GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  Objective = 112.412 (MAXimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 _C1          NS        .             .             .              .               -Inf     112.41200 payment2_1
                                            .             .                +Inf          +Inf     112.41200

     2 _C10         NS     100.00000        .          100.00000       60.00000          -Inf      70.80000 bond_1
                                           1.04030     100.00000           +Inf          +Inf          +Inf

     3 _C11         NS      90.00000        .           90.00000       70.00000          -Inf      91.81200 bond_2
                                           1.03000      90.00000           +Inf          +Inf          +Inf

     4 _C12         NS      70.00000        .           70.00000        8.80000          -Inf      50.60000 cashinhand_3
                                           1.01000      70.00000           +Inf          +Inf          +Inf

     5 _C13         NS      80.00000        .           80.00000      -32.41200          -Inf        .      cashinhand_4
                                           1.00000      80.00000           +Inf          +Inf          +Inf

     6 _C14         NS      60.00000        .           60.00000         .               -Inf     174.83000 payment1_1
                                          -1.04030      60.00000      100.00000          +Inf      70.80000 bond_1

     7 _C15         NS      70.00000        .           70.00000         .               -Inf     184.51200 payment1_2
                                          -1.03000      70.00000       90.00000          +Inf      91.81200 bond_2

     8 _C16         NS      50.00000        .           50.00000         .               -Inf     162.91200 payment1_3
                                          -1.01000      50.00000      111.20000          +Inf      50.60000 cashinhand_3

     9 _C17         NS      50.00000        .           50.00000         .               -Inf     162.41200 payment1_4
                                          -1.00000      50.00000      162.41200          +Inf        .      cashinhand_4

    10 _C2          NS        .             .             .              .               -Inf     112.41200 payment2_5
                                            .98039        .            51.00000          +Inf     162.41200 payment1_4

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   2

Problem:    
Objective:  Objective = 112.412 (MAXimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
    11 _C3          NS        .             .             .              .               -Inf     112.41200 payment3_1
                                            .             .                +Inf          +Inf     112.41200

    12 _C4          NS        .             .             .              .               -Inf     112.41200 payment3_2
                                          -1.03000        .            20.00000          +Inf      91.81200 bond_2

    13 _C5          NS        .             .             .              .               -Inf     112.41200 payment3_5
                                            .97115        .            52.00000          +Inf     162.91200 payment1_3

    14 _C6          NS        .             .             .              .               -Inf     112.41200 payment3_6
                                            .96154        .            52.00000          +Inf     162.41200 payment1_4

    15 _C7          NS        .             .             .              .               -Inf     112.41200 bond_0
                                           1.06090        .                +Inf          +Inf          +Inf

    16 _C8          NS        .             .             .              .               -Inf     112.41200 bond_3
                                          -1.01000        .            61.20000          +Inf      50.60000 cashinhand_3

    17 _C9          NS        .             .             .              .               -Inf     112.41200 bond_4
                                          -1.00000        .           112.41200          +Inf        .      cashinhand_4

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   3

Problem:    
Objective:  Objective = 112.412 (MAXimum)

   No. Column name  St      Activity      Obj coef   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 cashinhand_4 BS     112.41200       1.00000        .           112.21004        .             .      payment2_2
                                            .               +Inf      112.41200          +Inf          +Inf

     2 payment2_1   BS        .             .             .              .               -Inf     112.41200
                                            .               +Inf         .               +Inf     112.41200

     3 bond_1       BS      40.00000        .             .           -19.41748        .          112.41200 cashinhand_1
                                            .               +Inf      100.00000        .01010     112.81600 payment3_3

     4 cashinhand_1 NL        .             .             .           -19.80198          -Inf     112.41200 bond_2
                                            .               +Inf       40.00000        .          112.41200 bond_1

     5 payment1_1   BS      60.00000        .             .         -6060.00000       -.01010     111.80600 payment3_3
                                            .               +Inf       60.00000          +Inf          +Inf

     6 bond_0       BS        .             .             .              .               -Inf     112.41200
                                            .               +Inf         .               +Inf     112.41200

     7 bond_2       BS      20.00000        .             .        -11334.74747       -.00990     112.21400 cashinhand_2
                                            .               +Inf       60.40000        .          112.41200 cashinhand_1

     8 cashinhand_2 NL        .             .             .           -60.59406          -Inf     113.01188 cashinhand_3
                                           -.00990          +Inf       20.00000        .00990     112.21400 bond_2

     9 payment1_2   BS      70.00000        .             .            10.00000       -.00020     112.39800 payment2_3
                                            .               +Inf       70.00000          +Inf          +Inf

    10 payment2_2   NL        .             .             .           -40.80000          -Inf     112.82400 bond_1
                                           -.01010          +Inf       20.00000        .01010     112.21004 bond_2

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   4

Problem:    
Objective:  Objective = 112.412 (MAXimum)

   No. Column name  St      Activity      Obj coef   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
    11 payment3_2   BS        .             .             .              .               -Inf     112.41200
                                            .               +Inf         .               +Inf     112.41200

    12 bond_3       BS        .             .             .              .               -Inf     112.41200
                                            .               +Inf         .               +Inf     112.41200

    13 cashinhand_3 BS      61.20000        .             .            20.00000        .          112.41200 cashinhand_1
                                            .               +Inf       81.40000        .00980     113.01188 cashinhand_2

    14 payment1_3   BS      50.00000        .             .        -11191.20000       -.01000     111.91200 payment2_4
                                            .               +Inf       50.00000          +Inf          +Inf

    15 payment2_3   NL        .             .             .           -20.40000          -Inf     112.41600 bond_2
                                           -.00020          +Inf       61.20000        .00020     112.40000 cashinhand_3

    16 payment3_3   NL        .             .             .           -41.60000          -Inf     112.81600 bond_1
                                           -.00971          +Inf       62.40000        .00971     111.80600 payment1_1

    17 bond_4       BS        .             .             .              .               -Inf     112.41200
                                            .               +Inf         .               +Inf     112.41200

    18 payment1_4   BS      50.00000        .             .            50.00000          -Inf          -Inf
                                            .               +Inf       50.00000          +Inf          +Inf

    19 payment2_4   NL        .             .             .           -62.42400          -Inf     113.02400 cashinhand_3
                                           -.00980          +Inf       51.00000        .00980     111.91200 payment1_3

    20 payment3_4   NL        .             .             .           -20.80000          -Inf     112.61200 bond_2
                                           -.00962          +Inf       72.80000        .00962     111.71200 payment1_2

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   5

Problem:    
Objective:  Objective = 112.412 (MAXimum)

   No. Column name  St      Activity      Obj coef   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
    21 payment3_5   BS        .             .             .              .               -Inf     112.41200
                                            .               +Inf         .               +Inf     112.41200

    22 payment2_5   BS        .             .             .              .               -Inf     112.41200
                                            .               +Inf         .               +Inf     112.41200

    23 payment3_6   BS        .             .             .              .               -Inf     112.41200
                                            .               +Inf         .               +Inf     112.41200

    24 payment3_1   BS        .             .             .              .               -Inf     112.41200
                                            .               +Inf         .               +Inf     112.41200

End of report
